In [ ]:
import logging
!pip install pyspark
from pyspark.sql import SparkSession


### Create tabla delta y trigger para registrar los delta status
```sql
create table ct_status_delta
(
    ct_id              bigint      not null,
    status             varchar(20) not null,
    last_status_change timestamp,
    timestamp          timestamp default CURRENT_TIMESTAMP
);

alter table ct_status_delta
    owner to itoss;


CREATE OR REPLACE FUNCTION trg_ct_status_change()
RETURNS TRIGGER AS $$
DECLARE
    status_text varchar(20);

    BEGIN
    -- Determine status based on 'down' value

    IF NEW.down = TRUE THEN
        status_text := 'down';
    ELSE
        status_text := 'up';
    END IF;

    -- Check if 'down' or 'last_status_change' has changed
    IF NEW.down <> OLD.down OR NEW.last_status_change <> OLD.last_status_change THEN
        INSERT INTO ct_status_delta (ct_id, status, last_status_change)
        VALUES (NEW.id, status_text, NEW.last_status_change);
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;


CREATE TRIGGER ct_status_change_trigger
AFTER UPDATE OF down, last_status_change ON ct_status
FOR EACH ROW
EXECUTE FUNCTION trg_ct_status_change();
```

In [2]:
spark = (SparkSession.builder.appName("itoss-ai")
         .config("spark.jars","./postgresql-42.7.3.jar")
         .getOrCreate())

24/06/27 18:57:34 WARN Utils: Your hostname, notebook resolves to a loopback address: 127.0.1.1; using 192.168.101.13 instead (on interface wlp0s20f3)
24/06/27 18:57:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/06/27 18:57:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [19]:
jdbcUrl = "jdbc:postgresql://localhost:5432/itossconfig"
connection_properties = {
    "driver": "org.postgresql.Driver",
    "user": "itoss",
    "password": "admin"
}
query = "(select c.*, w.name as ctWorkgroup, l.name as ctLocation, case when cs.down then 'down' else 'up' end as ctStatus from ct c \
         inner join ct_status cs on c.id = cs.id \
         inner join workgroup w on c.workgroup_id = w.id \
         inner join location l on c.location_id = l.id \
         inner join ct_type ct on c.type_id = ct.id \
) as subquery"  
ctDf = spark.read.jdbc(url=jdbcUrl,table=query,properties=connection_properties)

In [20]:
ctDf.show()

+------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+------------+----------+------------+----------+----------+-----------+---------------------+---------------+-------+------------+--------------------------------+---------------------------+--------------+--------------------+----------+--------+
|    id|         created_by|       creation_date|    last_modified_by|  last_modified_date|          attributes|                 key|environment|                name|old_password|     state|collector_id|company_id|contact_id|location_id|monitoring_profile_id|support_user_id|type_id|workgroup_id|instrumentation_parameter_values|old_crypted_property_values|integration_id|         ctworkgroup|ctlocation|ctstatus|
+------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+------

In [21]:
metricsJdbcUrl = "jdbc:postgresql://localhost:5432/metrics"
connection_properties = {
    "driver": "org.postgresql.Driver",
    "user": "itoss",
    "password": "admin"
}
query = "(select * from linuxuptime) as subquery" 
uptimeDf = spark.read.jdbc(url=metricsJdbcUrl,table=query,properties=connection_properties)

In [22]:
uptimeDf.show()

[Stage 4:>                                                          (0 + 1) / 1]
Exception: java.lang.OutOfMemoryError thrown from the UncaughtExceptionHandler in thread "netty-rpc-env-timeout"
Exception in thread "Spark Context Cleaner" java.lang.OutOfMemoryError: Java heap space
Exception in thread "RemoteBlock-temp-file-clean-thread" java.lang.OutOfMemoryError: Java heap space
24/06/28 04:42:15 ERROR Executor: Exception in task 0.0 in stage 4.0 (TID 4)
java.lang.OutOfMemoryError: Java heap space
24/06/28 04:42:15 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 0.0 in stage 4.0 (TID 4),5,main]
java.lang.OutOfMemoryError: Java heap space
24/06/28 04:42:15 WARN TaskSetManager: Lost task 0.0 in stage 4.0 (TID 4) (notebook.bbrouter executor driver): java.lang.OutOfMemoryError: Java heap space

24/06/28 04:42:15 ERROR TaskSetManager: Task 0 in stage 4.0 failed 1 times; aborting job
ERROR:py4j.java_gateway:An error occurred whil